In [177]:

import pandas as pd
import numpy as np
import main as fca
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics
from importlib import reload
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import lazy_pipeline as lpipe
import folium
from folium.plugins import HeatMap
import plotly.express as px
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,plot_roc_curve,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_columns', 32)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [178]:
#data https://www.kaggle.com/datasets/joshmcadams/oranges-vs-grapefruit
df = pd.read_csv('citrus.csv')
df.head()

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9


In [179]:
df.isnull().sum().sort_values(ascending=False)

name        0
diameter    0
weight      0
red         0
green       0
blue        0
dtype: int64

In [180]:
df.isnull().any()

name        False
diameter    False
weight      False
red         False
green       False
blue        False
dtype: bool

In [181]:
df.dtypes

name         object
diameter    float64
weight      float64
red           int64
green         int64
blue          int64
dtype: object

In [182]:
df['name'].unique()

array(['orange', 'grapefruit'], dtype=object)

In [183]:
df['name'] = df['name'].replace(['orange'], 0)
df['name'] = df['name'].replace(['grapefruit'], 1)

In [184]:
df.head()

,name,diameter,weight,red,green,blue
0,0,2.96,86.76,172,85,2
1,0,3.91,88.05,166,78,3
2,0,4.42,95.17,156,81,2
3,0,4.47,95.60,163,81,4
4,0,4.48,95.76,161,72,9


In [185]:
set(df['name'])


{0, 1}

In [186]:
#df = df.rename(columns={"class": "target"})

In [187]:
df.head()

,name,diameter,weight,red,green,blue
0,0,2.96,86.76,172,85,2
1,0,3.91,88.05,166,78,3
2,0,4.42,95.17,156,81,2
3,0,4.47,95.60,163,81,4
4,0,4.48,95.76,161,72,9


In [188]:
df = df.dropna()
df.head()
cols = list(df.columns)
cols.remove('name')
X = df.loc[:,cols]
y = df.name
X.shape,y.shape

((10000, 5), (10000,))

In [189]:
set(y)

{0, 1}

In [190]:
X = X.sample(frac=1, random_state=42)  # Shuffle X
y = y[X.index]

In [191]:
X=X.drop(X.index[900:])
y = y.drop(y.index[900:])

In [192]:
y.value_counts()

1    461
0    439
Name: name, dtype: int64

In [193]:
y[1:15]

4684    0
1731    0
4742    0
4521    0
6340    1
576     0
5202    1
6363    1
439     0
2750    0
7487    1
5272    1
5653    1
3999    0
Name: name, dtype: int64

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [221]:
reload(fca)

<module 'main' from 'd:\\HSE\\OSDA\\LazyFCA\\main.py'>

In [222]:
fca.fitClassifier(X_train, y_train)

In [229]:
y_pred = fca.predict(X_test,'plus')

In [230]:
results = {'accuracy': [], 'precision': [], 'recall': []}
results['accuracy'].append(metrics.accuracy_score(y_test, y_pred))
results['precision'].append(metrics.precision_score(y_test, y_pred))
results['recall'].append(metrics.recall_score(y_test, y_pred))

print('Accuracy:', pd.DataFrame(results)['accuracy'])

Accuracy: 0    0.677778
Name: accuracy, dtype: float64


In [231]:

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ]

log_cols=["Classifier", "Accuracy", "Precision", "Recall"]
log = pd.DataFrame(columns=log_cols)


In [232]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    
    train_predictions = clf.predict(X_test)
    prec = precision_score(y_test, train_predictions)
    
    
    train_predictions = clf.predict(X_test)
    rec = recall_score(y_test, train_predictions)
      
    
    train_predictions = clf.predict_proba(X_test)
    
    
    log_entry = pd.DataFrame([[name, acc, prec, rec]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [233]:
print(log)

                   Classifier  Accuracy  Precision    Recall
0      DecisionTreeClassifier  0.903704   0.913669  0.900709
0      RandomForestClassifier  0.918519   0.934307  0.907801
0          AdaBoostClassifier  0.922222   0.947761  0.900709
0  GradientBoostingClassifier  0.911111   0.927007  0.900709


In [234]:
f_acc,f_pre,f_rec = results['accuracy'][0],results['precision'][0],results['recall'][0]
fcaa_row = {'Classifier': 'FCA', 'Accuracy': f_acc, 'Precision': f_pre,'Recall':f_rec}
log = log.append(fcaa_row, ignore_index = True)
  
display(log)

,Classifier,Accuracy,Precision,Recall
0,DecisionTreeClassifier,0.903704,0.913669,0.900709
1,RandomForestClassifier,0.918519,0.934307,0.907801
2,AdaBoostClassifier,0.922222,0.947761,0.900709
3,GradientBoostingClassifier,0.911111,0.927007,0.900709
4,FCA,0.677778,0.707692,0.652482


In [235]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

log.sort_values(by = 'Accuracy', ascending = False)
fig = px.bar(data_frame = log, x = 'Accuracy', y = 'Classifier', color = 'Accuracy', template = 'ggplot2', title = 'Models Comparison')
fig.show()